# Populate Silver

## Load Schools

In [1]:
-- Populate schools
INSERT INTO dim_school ( dsId, deReferenceNumber, schoolName, country )
SELECT
    ROW_NUMBER() OVER ( ORDER BY DE_Reference_Number ),
    DE_Reference_Number,
    School_Name,
    'Northern Ireland'
FROM
    (
    SELECT DISTINCT
        DE_Reference_Number,
        School_Name
    FROM bronze_attendance
    WHERE DE_Reference_Number IS NOT NULL
    ) x
ORDER BY 1;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
SELECT * FROM dim_school;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 3, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 4 fields>

In [3]:
SELECT * FROM bronze_attendance LIMIT 100;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 4, Finished, Available, Finished)

<Spark SQL result set with 100 rows and 8 fields>

In [4]:
SELECT * FROM fact_attendance LIMIT 100;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 5 fields>

In [5]:
DROP TABLE IF EXISTS fact_attendance;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
CREATE TABLE IF NOT EXISTS fact_attendance (
    faId                INT NOT NULL,
    dsId                INT NOT NULL,
    stId                INT NOT NULL,
    yearId              INT NOT NULL,
    percentAttendance   DECIMAL(5,2) NOT NULL
    --...
    );

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
-- Populate fact_attendance
INSERT INTO fact_attendance ( faId, dsId, stId, yearId, percentAttendance )
WITH cte AS (
SELECT
    REPLACE( REPLACE( sourceFile, 'Schools-Level-attendance-Tables-', '' ), '.xlsx', '' ) cleanYear,
    s.dsId,
    st.stId,
    ba.`%_attendance`
FROM bronze_attendance ba
    INNER JOIN dim_school s ON ba.DE_Reference_Number = s.deReferenceNumber
        AND ba.School_Name = s.schoolName 
    INNER JOIN dim_schoolType st ON ba.sourceTab = st.schoolType
WHERE DE_Reference_Number IS NOT NULL
)
SELECT 
    ROW_NUMBER() OVER( ORDER BY dsId, stId, yearId ),
    dsId,
    stId,
    y.yearId,
    `%_attendance`
FROM cte c
    INNER JOIN dim_year y ON c.cleanYear = y.yearId;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
SELECT * FROM fact_attendance LIMIT 100;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 9, Finished, Available, Finished)

<Spark SQL result set with 100 rows and 5 fields>

In [9]:
-- CREATE VIEW vw_fact_attendance AS
SELECT
    s.deReferenceNumber,
    s.schoolName,
    y.yearDesc,
    f.percentAttendance,
    st.schoolType
FROM fact_attendance f
    INNER JOIN dim_school s ON f.dsId = s.dsId
    INNER JOIN dim_schoolType st ON f.stId = st.stId
    INNER JOIN dim_year y ON f.yearId = y.yearId
LIMIT 100;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 10, Finished, Available, Finished)

<Spark SQL result set with 100 rows and 5 fields>

In [10]:
CREATE OR REPLACE VIEW vw_fact_attendance_primary
AS
SELECT
    s.deReferenceNumber,
    s.schoolName,
    y.yearDesc,
    f.percentAttendance
FROM fact_attendance f
    INNER JOIN dim_school s ON f.dsId = s.dsId
    INNER JOIN dim_schoolType st ON f.stId = st.stId
    INNER JOIN dim_year y ON f.yearId = y.yearId
WHERE st.schoolType = 'Primary';

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
CREATE OR REPLACE VIEW vw_fact_attendance_post_primary
AS
SELECT
    s.deReferenceNumber,
    s.schoolName,
    y.yearDesc,
    f.percentAttendance
FROM fact_attendance f
    INNER JOIN dim_school s ON f.dsId = s.dsId
    INNER JOIN dim_schoolType st ON f.stId = st.stId
    INNER JOIN dim_year y ON f.yearId = y.yearId
WHERE st.schoolType = 'Post-Primary';

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
CREATE OR REPLACE VIEW vw_fact_attendance_special
AS
SELECT
    s.deReferenceNumber,
    s.schoolName,
    y.yearDesc,
    f.percentAttendance
FROM fact_attendance f
    INNER JOIN dim_school s ON f.dsId = s.dsId
    INNER JOIN dim_schoolType st ON f.stId = st.stId
    INNER JOIN dim_year y ON f.yearId = y.yearId
WHERE st.schoolType = 'Special';

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
SELECT * FROM vw_fact_attendance_primary WHERE yearDesc = '2021/22' LIMIT 100;
SELECT * FROM vw_fact_attendance_post_primary WHERE yearDesc = '2021/22' LIMIT 100;
SELECT * FROM vw_fact_attendance_special WHERE yearDesc = '2021/22' LIMIT 100;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 16, Finished, Available, Finished)

<Spark SQL result set with 100 rows and 4 fields>

<Spark SQL result set with 100 rows and 4 fields>

<Spark SQL result set with 39 rows and 4 fields>

In [14]:
SELECT * FROM vw_fact_attendance_primary WHERE yearDesc = '2023/24' LIMIT 100;
SELECT * FROM vw_fact_attendance_post_primary WHERE yearDesc = '2023/24' LIMIT 100;
SELECT * FROM vw_fact_attendance_special WHERE yearDesc = '2023/24' LIMIT 100;

StatementMeta(, 65b09621-808b-4c16-8a11-9ad9c0b53a2b, 19, Finished, Available, Finished)

<Spark SQL result set with 100 rows and 4 fields>

<Spark SQL result set with 100 rows and 4 fields>

<Spark SQL result set with 39 rows and 4 fields>